<a href="https://colab.research.google.com/github/dabin3178/Korean-Newords-NLP/blob/main/%EB%B3%80%EC%88%98%EC%B6%94%EC%B6%9C(0819).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install soynlp

     |████████████████████████████████| 416 kB 12.5 MB/s 


In [ ]:
import re

def text_preprocess(x): # 공백도 제외
    text=[]
    a = re.sub('[^가-힣\\s]', '',x)
    for j in a.split():
        text.append(j)
    return ' '.join(text)

In [ ]:
import pandas as pd
eta_secret_labeled = pd.read_csv("/content/drive/MyDrive/Project_FINAL/raw_data/eta_비밀.csv")
eta_sisa_labeled = pd.read_csv("/content/drive/MyDrive/Project_FINAL/raw_data/df_에타_sisa.csv")
eta_global_labeled = pd.read_csv("/content/drive/MyDrive/Project_FINAL/raw_data/Globalfree.csv")
deoku_labeled = pd.read_csv("/content/drive/MyDrive/Project_FINAL/raw_data/deoku.csv")
ppomppu_labeled = pd.read_csv("/content/drive/MyDrive/Project_FINAL/raw_data/ppomppu1.csv")
ppomppu2_labeled = pd.read_csv("/content/drive/MyDrive/Project_FINAL/raw_data/ppomppu2.csv")
youtube_labeled = pd.read_csv("/content/drive/MyDrive/Project_FINAL/raw_data/유튜브15개.csv")
navernews_labeled = pd.read_csv("/content/drive/MyDrive/Project_FINAL/raw_data/navernews.csv")
nate_pann_labeled = pd.read_csv("/content/drive/MyDrive/Project_FINAL/raw_data/네이트판_통합.csv")
dc_inside_labeled = pd.read_csv("/content/drive/MyDrive/Project_FINAL/raw_data/df_DC_total.csv")

labeled_data_df = pd.concat([eta_secret_labeled, eta_sisa_labeled],ignore_index=True)
labeled_data_df = pd.concat([labeled_data_df, eta_global_labeled],ignore_index=True)
labeled_data_df = pd.concat([labeled_data_df, deoku_labeled],ignore_index=True)
labeled_data_df = pd.concat([labeled_data_df, ppomppu_labeled],ignore_index=True)
labeled_data_df = pd.concat([labeled_data_df, ppomppu2_labeled],ignore_index=True)
labeled_data_df = pd.concat([labeled_data_df, youtube_labeled],ignore_index=True)
labeled_data_df = pd.concat([labeled_data_df, navernews_labeled],ignore_index=True)
labeled_data_df = pd.concat([labeled_data_df, nate_pann_labeled],ignore_index=True)
labeled_data_df = pd.concat([labeled_data_df, dc_inside_labeled],ignore_index=True)

for column in labeled_data_df.columns:
    if column != 'comment':
        labeled_data_df = labeled_data_df.drop(column, axis=1)

labeled_data_df.to_csv("/content/drive/MyDrive/Project_FINAL/total_data.csv", encoding='utf-8-sig', index=False)

labeled_data_df

,comment
0,1
1,1
2,4
3,4
4,0
...,...
1996139,야갤하는 여자는 도대체 어떤 인생을 사는 걸까
1996140,섹스
1996141,pp
1996142,흐읏흥


In [ ]:
from soynlp import DoublespaceLineCorpus
from soynlp.word import WordExtractor
import pandas as pd

df = labeled_data_df # pd.read_csv("/content/drive/MyDrive/Project_FINAL/total_data.csv", encoding="utf-8", engine="python") # raw data 불러오기

train_corpus = "" # raw data를 쭉이을 빈 문자열

df['comment'] = df['comment'].apply(lambda x: str(x))
df['process'] = df['comment'].apply(lambda x: text_preprocess(x)) # 불필요한 문자 제거

for i in range(len(df['process'])):
    train_corpus += df['process'][i] + '\n' # 문자열을 쭉 이음.

for i in range(2):
    train_corpus += train_corpus + '\n'

In [ ]:
with open("/content/drive/MyDrive/Project_FINAL/train_corpus.txt", 'w') as file: # txt 파일로 저장
    file.write(train_corpus)

In [ ]:
train_corpus = DoublespaceLineCorpus("/content/drive/MyDrive/Project_FINAL/train_corpus.txt") # 저장한 txt 파일로 변환
len(train_corpus)

3060435

In [ ]:
# 코퍼스로 학습
word_extractor = WordExtractor()
word_extractor.train(train_corpus)
word_score_table = word_extractor.extract()

training was done. used memory 2.600 Gb
all cohesion probabilities was computed. # words = 518782
all branching entropies was computed # words = 1595913
all accessor variety was computed # words = 1595913


In [ ]:
#후보 신조어 불러오기
partial_words_df = pd.read_csv("/content/drive/MyDrive/Project_FINAL/partial_total.csv")

partial_words_df

,partial_words,freq,freq_ratio
0,하는,129084,0.270207
1,부탁해,45032,0.094264
2,댓글부탁,44000,0.092104
3,댓글부,43948,0.091995
4,있는,43772,0.091627
...,...,...,...
23210,안한거지,64,0.000134
23211,특히나,64,0.000134
23212,뉴온달,64,0.000134
23213,수집완료,64,0.000134


In [ ]:
word_score_table['야알못'].leftside_frequency # 단어가 잘 되었는지 테스트, 단어는 알아서 바꾸세요.

1432

In [ ]:
# KeyError가 나지 않는 부분 단어들만 따로 저장.

partial_words_extracted = []
freq_extracted = []
freq_ratio_extracted = []

for i in range(len(partial_words_df['partial_words'])):
    try:
        word_score_table[partial_words_df['partial_words'][i]]
        partial_words_extracted.append(partial_words_df['partial_words'][i])
        freq_extracted.append(partial_words_df['freq'][i])
        freq_ratio_extracted.append(partial_words_df['freq_ratio'][i])
    except KeyError:
        pass

partial_data = {
    'partial_words': partial_words_extracted,
    'freq': freq_extracted,
    'freq_ratio': freq_ratio_extracted
}

partial_words_df = pd.DataFrame(partial_data)

partial_words_df.shape

(23215, 3)

In [ ]:
partial_words_df

,partial_words,freq,freq_ratio
0,하는,129084,0.270207
1,부탁해,45032,0.094264
2,댓글부탁,44000,0.092104
3,댓글부,43948,0.091995
4,있는,43772,0.091627
...,...,...,...
23210,안한거지,64,0.000134
23211,특히나,64,0.000134
23212,뉴온달,64,0.000134
23213,수집완료,64,0.000134


In [ ]:
# 필요한 변수들 계산
partial_words_df['length'] = partial_words_df['partial_words'].apply(lambda x: len(x))
partial_words_df['cohesion_forward'] = partial_words_df['partial_words'].apply(lambda x: word_score_table[x].cohesion_forward)
partial_words_df['left_branching_entropy'] = partial_words_df['partial_words'].apply(lambda x: word_score_table[x].left_branching_entropy)
partial_words_df['right_branching_entropy'] = partial_words_df['partial_words'].apply(lambda x: word_score_table[x].right_branching_entropy)
partial_words_df['left_accessor_variety'] = partial_words_df['partial_words'].apply(lambda x: word_score_table[x].left_accessor_variety)
partial_words_df['starts_ratio'] = partial_words_df['partial_words'].apply(lambda x: word_score_table[x].leftside_frequency)

for i in range(len(partial_words_df['starts_ratio'])):
    partial_words_df['starts_ratio'][i] = (partial_words_df['starts_ratio'][i] / partial_words_df['freq'][i]) * 100

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
partial_words_df['label'] = [0] * len(partial_words_df['partial_words'])

partial_words_df

,partial_words,freq,freq_ratio,length,cohesion_forward,left_branching_entropy,right_branching_entropy,left_accessor_variety,starts_ratio,label
0,하는,129084,0.270207,2,0.271166,5.339681,5.138724,919,29,0
1,부탁해,45032,0.094264,3,0.090885,0.187021,3.773582,57,1,0
2,댓글부탁,44000,0.092104,4,0.939333,1.945910,0.014704,7,99,0
3,댓글부,43948,0.091995,3,0.910643,2.484907,0.004263,12,100,0
4,있는,43772,0.091627,2,0.311924,3.351824,5.005769,376,77,0
...,...,...,...,...,...,...,...,...,...,...
23210,안한거지,64,0.000134,4,0.079161,1.927392,2.252728,9,100,0
23211,특히나,64,0.000134,3,0.065477,2.512659,2.084430,13,100,0
23212,뉴온달,64,0.000134,3,0.082026,2.014036,1.799959,10,100,0
23213,수집완료,64,0.000134,4,0.096823,0.000000,0.000000,0,100,0


In [ ]:
partial_words_df.to_csv("/content/drive/MyDrive/Project_FINAL/variables.csv", encoding='utf-8-sig', index=False)

#----------------절취선-------------------

In [ ]:
eta_secret_labeled = pd.read_csv("/content/drive/MyDrive/Project_FINAL/labeled_variables/variables_eta비.csv")
eta_sisa_labeled = pd.read_csv("/content/drive/MyDrive/Project_FINAL/labeled_variables/variables_eta시사.csv")
eta_global_labeled = pd.read_csv("/content/drive/MyDrive/Project_FINAL/labeled_variables/variables_eta글자게.csv")
deoku_labeled = pd.read_csv("/content/drive/MyDrive/Project_FINAL/labeled_variables/variables_deoku.csv")
ppomppu_labeled = pd.read_csv("/content/drive/MyDrive/Project_FINAL/labeled_variables/variables_ppom1_label.csv")
youtube_labeled = pd.read_csv("/content/drive/MyDrive/Project_FINAL/labeled_variables/variables_youtube.csv")
navernews_labeled = pd.read_csv("/content/drive/MyDrive/Project_FINAL/labeled_variables/variables_네이버뉴스.csv")
nate_pann_labeled = pd.read_csv("/content/drive/MyDrive/Project_FINAL/labeled_variables/nate_variables_labeled.csv")
dc_inside_labeled = pd.read_csv("/content/drive/MyDrive/Project_FINAL/labeled_variables/dc_variables_labeled.csv")

labeled_data_df = pd.concat([eta_secret_labeled, eta_sisa_labeled],ignore_index=True)
labeled_data_df = pd.concat([labeled_data_df, eta_global_labeled],ignore_index=True)
labeled_data_df = pd.concat([labeled_data_df, deoku_labeled],ignore_index=True)
labeled_data_df = pd.concat([labeled_data_df, ppomppu_labeled],ignore_index=True)
labeled_data_df = pd.concat([labeled_data_df, youtube_labeled],ignore_index=True)
labeled_data_df = pd.concat([labeled_data_df, navernews_labeled],ignore_index=True)
labeled_data_df = pd.concat([labeled_data_df, nate_pann_labeled],ignore_index=True)
labeled_data_df = pd.concat([labeled_data_df, dc_inside_labeled],ignore_index=True)


labeled_data_df = labeled_data_df.drop(['Unnamed: 0'], axis=1)

labeled_dict = {k: 0 for k in labeled_data_df['partial_words']}

for i in range(len(labeled_data_df)):
    labeled_dict[labeled_data_df['partial_words'][i]] = labeled_data_df['label'][i]
    
for i in range(0, len(partial_words_df)):
    try:
        partial_words_df['label'][i] = labeled_dict[partial_words_df['partial_words'][i]]
    except KeyError:
        partial_words_df['label'][i] = -1
        
partial_words_df.to_csv("/content/drive/MyDrive/Project_FINAL/labeled.csv", encoding='utf-8-sig', index=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
labeled_data_df.to_csv("/content/drive/MyDrive/Project_FINAL/total_labeled_data.csv", encoding="utf-8-sig", index=False)